## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [1]:
# Установить можно через pip.
from natasha import NamesExtractor

In [2]:
sents = open('sents.txt').read().splitlines()

In [3]:
extractor = NamesExtractor()

In [4]:
extractor('\n'.join(sents))

Matches('Напомним, что премьер-министр России Дмитрий Медведев подписал изменения в Правилах дорожного движения.\nВ секте Асахары «министр» Ниими слыл страшным человеком.\nС тайной полицией Асахары был связан еще один из семи этапированных накануне смертников - Кийохиде Хаякава, «министр созидания».\nПо словам Косуги, накануне «министр» Хаякава в личной беседе поблагодарил его за помощь «Аум» и попросил сделать кое-что для «спасения человечества», а именно - убить шефа полиции Японии.\nКак только что сообщило немецкое государственное информационное агентство "Deutsche Welle", видный немецкий политик от СДПГ, в прошлом премьер-министр земли Бранденбург Маттиас Платцек, в интервью радиостанции "Deutschlandfunk" в пятницу 16 марта заявил, что по его сведениям лаборатория, которая занималась исследованиями боевых отравляющих веществ в советское время находилась на территории нынешнего Узбекистана.\nМежду тем вице-премьер РМ по евроинтеграции Юрий Лянкэ заявил накануне на румынском канале Digi24, что объединение с Румынией - тема, которая становится все более важной в Республике Молдова.\nПремьер-министр Израиля Биньямин Нетаньяху и мэр Иерусалима Нир Баркат приняли решение создать правительственную комиссию под руководством министра регионального сотрудничества Цахи Ханегби с участием всех соответствующих сторон для разработки решения вопроса о муниципальных налогах на недвижимость, принадлежащую христианским Церквам и не являющуюся молельными домами.\nВ МИД Израиля «НГР» сообщили, что в рамках дискуссии между администрацией главы правительства и муниципалитетом Иерусалима по этому вопросу премьер-министр и мэр Иерусалима договорились о том, что профессиональная группа во главе с министром Ханегби, в которую войдут представители мэрии израильской столицы, а также министерств финансов, иностранных дел и внутренних дел выработает согласованное решение относительно муниципальных налогов с церковной недвижимости, не являющейся молельными домами.\nКроме того, в соответствии с поступившей от глав Церквей просьбой начать переговоры о вопросах, связанных с продажей земельных участков в Иерусалиме, премьер-министр Нетаньяху поручил министру Ханегби изучить этот вопрос.\nПремьер-министр Великобритании Тереза Мэй в качестве «жесткого ответа» в связи с покушением на двойного агента Сергея Скрипаля и его дочь Юлию вышлет из страны 23 российских дипломата.\nПремьер-министр Алексис Ципрас провел телефонные консультации с председателем Еврокомиссии Жан-Клодом Юнкером и главой Европейского центрального банка (ЕЦБ) Марио Драги.\nПремьер пообещал, что покинет пост, если греки согласятся с условиями, выдвигаемыми кредиторами.\nМинистр финансов Греции Янис Варуфакис еще днем заявил: «Греция не будет переводить платеж МВФ сегодня».\n«Цель референдума - продолжение переговоров», - уточнил премьер.\nКак отметил экс-министр экономики и финансов Польши Гжегож Колодко (иностранный член Российской академии наук), именно отсутствие грамотной экономической стратегии в РФ привело к печальным результатам.\nПремьер Зубков, решая судьбу Чаянды, был поставлен перед выбором: поступить ли по закону или исходя из политической целесообразности.\nПозже ту же версию подтвердил в телефонном разговоре с Владимиром Путиным премьер-министр Великобритании Дэвид Кэмерон, ссылаясь на данные своей разведки.\nИз Foreign Office поступила информация, подтверждающая факт обмена данными о крушении А321 на Синае между британским премьер-министром и российским президентом.\nИ сообщил, что начальник Генштаба и министр обороны уже получили соответствующие указания.\nВ список среди прочих попали премьер-министр Дмитрий Медведев и все члены кабинета министров, а также 43 сотрудника администрации президента РФ, включая пресс-секретаря Дмитрия Пескова, главу администрации Антона Вайно, советников и помощников главы государства.\nВ понедельник спецпредставитель президента Грузии, бывший министр иностранных дел Давид Бакрадзе выехал в Брюссель на заседание Совета НАТО в формате «26 плюс 1» (чл

Видно, что извлекается не очень хорошо. К тому же в реальной задаче нам потребуется извлекать ещё какие-то аттрибуты сущностей и группировать их в факты. И не обязательно этих типов. Поэтому полезно разобраться с самим парсером yargy.

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

In [29]:
from yargy import Parser, rule
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display


Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'премьер министр',
    'президент'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [32]:
for sent in sents[:100]:
    
    for match in parser.findall(sent):
        display(match.fact)
    print('---------------')

In [13]:
matches = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [22]:
matches

[Person(position='президент',
        name=Name(first='гурбангул',
                  last='бердымухамедов')), Person(position='президент',
        name=Name(first='виктор',
                  last='ющенко')), Person(position='президент',
        name=Name(first='владимир',
                  last='путин'))]

In [15]:
len(matches)

3

Чтобы проверить какие морфологические тэги приписываются словам, можно использовать MorphTokenizer.

In [30]:
tokenizer = MorphTokenizer()

In [31]:
list(tokenizer('Медведев'))

[MorphToken('Медведев',
            [0, 8),
            'RU',
            [Form('медведев', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])]

## Задача на семинар:
Доработать правила так, чтобы извлекалось как можно больше правильных фактов.

Документация yargy вот тут - https://yargy.readthedocs.io/ru/latest/reference.html